## Importer le modèle

In [ ]:
from keras.models import load_model
import numpy as np
import os
model = load_model('GoodModel/models1.h5')
# summarize model.
model.summary()

## Importer l'image

In [ ]:
import rasterio
#Mettre les chemins d'accès aux bandes utilisées par l'entrainement du
#modèle utilisé dans le bon ordre dans le tableau ci-dessous.
bandsvv = ['S1Fall/fa_vv.tiff',
          'S1Winter/wi_vv.tiff',
          'S1Spring/sp_vv.tiff',
          'S1Summer/su_vv.tiff']
bandsvh = ['S1Fall/fa_vh.tiff',
          'S1Winter/wi_vh.tiff',
          'S1Spring/sp_vh.tiff',
          'S1Summer/su_vh.tiff']

In [ ]:
IMAGE_PIXEL = 30
IMAGE_WIDTH = 838
IMAGE_HEIGHT = 560
BAND_WIDTH = 8

In [ ]:
def reshape(image, width):
    """Transforme une image raster pour pouvoir être utilisable par le réseau de neurones"""
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            for k in range(width):
                reshaped_cell.append(image[k][i][j] / 65535)
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

In [ ]:
def split_band(band, width, height, pixel_length):
    """Split une bande de Sentinel-1 en bande de plus petites images"""
    result = []
    for x in range(height):
        columns = []
        for y in range(width):
            lines = []
            for i in range(pixel_length):
                cells = []
                for j in range(pixel_length):
                    if (i + IMAGE_PIXEL * x) < len(band) and  (j + IMAGE_PIXEL * y) < len(band[0]):
                        cells.append(band[i + IMAGE_PIXEL * x][j + IMAGE_PIXEL * y])
                    else:
                        cells.append(0)
                lines.append(cells)
            columns.append(lines)
        result.append(columns)
    return result

In [ ]:
def get_vectors(gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left):
    """Forme les vecteurs moyens entre 4 coordonnées GroundControlPoint"""
    norm_v1 = (gcp_top_left.col - gcp_top_right.col)
    v1 = np.array([(gcp_top_left.x - gcp_top_right.x + gcp_bottom_left.x - gcp_bottom_right.x) / (2 * norm_v1),
         (gcp_top_left.y - gcp_top_right.y + gcp_bottom_left.y - gcp_bottom_right.y) / (2 * norm_v1)])
    norm_v2 = (gcp_bottom_right.row - gcp_top_right.row)
    v2 = np.array([(gcp_bottom_right.x - gcp_top_right.x + gcp_bottom_left.x - gcp_top_left.x) / (2 * norm_v2),
         (gcp_bottom_right.y - gcp_top_right.y + gcp_bottom_left.y - gcp_top_left.y ) / (2 * norm_v2)])
    return v1, v2

In [ ]:
from sympy import symbols, Eq, solve
def get_offset(gcps, x_min, y_min):
    """Retourne déplacements à effectuer pour avoir la position d'une coordonnée particulière (environ)"""
    gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left = None, None, None, None
    for gcp in gcps:
        if gcp.row == 0.0 and gcp.col == 0.0:
            gcp_top_right = gcp
        elif gcp.row == 0.0 and gcp_top_left == None:
            gcp_top_left = gcp
        elif gcp.col <= 0.0 and gcp_bottom_right == None:
            gcp_bottom_right = gcp
        elif gcp_bottom_right != None:
            gcp_bottom_left = gcp
            break
    v1, v2 = get_vectors(gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left)
    origin = np.array([gcp_top_right.x, gcp_top_right.y])
    u1, u2 = symbols('u1 u2')
    eq1 = Eq(gcp_top_right.x - u1*v1[0] - u2*v2[0] - x_min, 0)
    eq2 = Eq(gcp_top_right.y - u1*v1[1] - u2*v2[1] - y_min, 0)
    sol = solve((eq1, eq2),(u1, u2))

    return round(sol[u1]), round(sol[u2])

In [ ]:
def translate_image(image, offset_x, offset_y):
    """Fait correspondre la position [0][0] de notre image à la position [offset_y][offset_x], toutes positions en dehors de
    l'image de base seront mises à 0."""
    result = []
    for i in range(len(image)):
        if 0 <= (i + offset_y) < len(image):
            result_row = []
            for j in range(len(image[0])):
                if 0 <= (j + offset_x) < len(image[0]):
                    result_row.append(image[i + offset_y][j + offset_x])
                else:
                    result_row.append(0)
            result.append(result_row)
        else:
            result.append([0 for i in range(len(image[0]))])
    return np.array(result, dtype=np.uint16)

In [ ]:
from osgeo import gdal
def create_gcps(list_gcps):
    """Fait une approximation des gcps en moyennant les gcps de toutes les bandes de saisons (prend comme point de départ le
    plus petit x et le plus petit y trouvé)"""
    firsts_x = []
    firsts_y = []
    for band in list_gcps:
        firsts_x.append(band[0].x)
        firsts_y.append(band[0].y)
    x_min = min(firsts_x)
    y_min = min(firsts_y)
    gcps = [rasterio.control.GroundControlPoint(row=0.0, col=0.0, x=x_min, y=y_min, id='1')]
    for i in range(1, len(list_gcps[0])):
        previous_x_average = 0
        previous_y_average = 0
        actual_x_average = 0
        actual_y_average = 0
        for band in list_gcps:
            previous_x_average += band[i-1].x
            previous_y_average += band[i-1].y
            actual_x_average += band[i].x
            actual_y_average += band[i].y
        x = gcps[i-1].x + (actual_x_average - previous_x_average) / len(list_gcps)
        y = gcps[i-1].y + (actual_y_average - previous_y_average) / len(list_gcps)  
        gcps.append(rasterio.control.GroundControlPoint(row=list_gcps[0][i].row, col=list_gcps[0][i].col,
                                                        x=x, y=y, id=str(i+1)))
    return gcps

In [ ]:
def make_blue(vv, vh):
    #Sources https://stackoverflow.com/questions/3950372/round-with-integer-division
    return (vv + vh // 2) // vh 

## Faire la prédiction

In [ ]:
def predict(bandsvv, bandsvh, model, output_path="output", output_name="prédiction", make_blue=False):
    """Créée l'image de sortie en noir et blanc"""
    print("Has started")
    images = []
    width = None
    height = None
    crs = None
    transform = None
    dtype = None
    gcps = None
    for i in range(len(bandsvv)):
        imagevv = rasterio.open(bandsvv[i])
        if width == None:
            width = imagevv.width
            height = imagevv.height
            transform = imagevv.transform
            dtype = imagevv.dtypes[0]
            gcps, crs = imagevv.gcps
        images.append(split_band(imagevv.read(1), IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_PIXEL))
        imagevh = rasterio.open(bandsvh[i])        
        images.append(split_band(imagevh.read(1), IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_PIXEL))
        if make_blue:
            images.append(split_band(make_blue(imagevv.read(1), imagevh.read(1))))
        print("Fin", bandsvv[i], "et", bandsvh[i])        
    prediction = []
    prediction_len = set()
    for x in range(IMAGE_HEIGHT):
        print(x + 1, "/", IMAGE_HEIGHT)
        row_prediction = []
        for y in range(IMAGE_WIDTH):
            smaller_image = []
            for image in images:
                smaller_image.append(image[x][y])
            value = model.predict(np.expand_dims(reshape(smaller_image, BAND_WIDTH), axis=0))
            prediction_len.add(value[0][0] * 65535)
            for i in range(IMAGE_PIXEL):
                row_prediction.append(value[0][0] * 65535)
                
        for i in range(IMAGE_PIXEL):
            prediction.append(np.array(row_prediction).astype('uint16'))
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    output = rasterio.open(output_path + '/' + output_name + '.tiff', 'w', driver='Gtiff', 
                          width=width, height=height, count=1, crs=crs, gcps=gcps,
                          transform=transform, 
                          dtype=dtype)
    output.write(prediction, 1)
    output.close()
    print("Le fichier de prédiction a été créé.")

In [ ]:
predict(bandsvv, bandsvh, model=model, output_path="output", output_name="prédictionS1AllSeasons")